# 提供高级模型的可解释性

1. RF、DT、ET、XGBoost、LightGBM模型的特征重要性。
2. XGBoost、LightGBM模型的决策过程

需要动手能力比较强的同学，需要安装GraphViz。
1. 官网下载：https://graphviz.org/download/
2. 安装过程记得添加环境变量到当前用户
3. 安装python接口：`pip install graphviz`

可以参考博客：https://blog.csdn.net/chenkfkevin/article/details/111443521

## LR模型输出公式

label = 2.9152857686530913  -0.205248 * prob02 +0.687838 * prob03 +0.432564 * prob04 -0.409445 * prob05 -0.386123 * prob06 -0.915558 * prob07 +0.375201 * prob08 +0.226856 * prob09 +2.639113 * prob10 +1.288628 * pred1

## 树模型输出FeatureImportance

1. XGBoost
2. LightGBM
3. ExtraTrees
4. RandomForest等

## 输出XGBoost和LightGBM决策过程


In [ ]:
from onekey_algo.custom.Manager import onekey_show

onekey_show('模型可解释性-SHAP')

In [ ]:
import os
import joblib
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from onekey_algo import get_param_in_cwd, init_CN
from onekey_algo.custom.components.comp1 import plot_feature_importance

init_CN()
COEF_THRESHOLD = 1e-6

model_root = r'C:\Users\onekey\Desktop\onekey_comp\comp9-Solutions\sol1. 传统组学-单中心-临床\models'

In [ ]:
save_dir = get_param_in_cwd('save_dir', 'viz')
os.makedirs(save_dir, exist_ok=True)
os.environ['PATH'] += ';' + os.path.join(os.environ['ONEKEY_HOME'], r'onekey_envs\Library\graphviz')
for pkl in os.listdir(model_root):
    try:
        model = joblib.load(os.path.join(model_root, pkl))
        save_name = os.path.splitext(pkl)[0]
        print(model.__class__.__name__, "，模型位置：", pkl)
        if 'xgboost' in pkl.lower():
            plot_feature_importance(model, feature_names=list(model.get_booster().get_score().keys()), show=False)
            plt.savefig(os.path.join(save_dir, f"{save_name}_fimp.svg"), bbox_inches='tight')
            xgb.plot_tree(model, save_name=os.path.join(save_dir, save_name))
        elif 'lightgbm' in pkl.lower():
            plot_feature_importance(model, feature_names=model.feature_name_, show=False)
            plt.savefig(os.path.join(save_dir, f"{save_name}_fimp.svg"), bbox_inches='tight')
            lgb.plot_tree(model, orientation='vertical', save_name=os.path.join(save_dir, save_name), dpi=300)
        elif 'lr' in pkl.lower():
            feature_names = list(model.feature_names_in_)
            feat_coef = [(feat_name, coef) for feat_name, coef in zip(feature_names, model.coef_[0]) 
                         if COEF_THRESHOLD is None or abs(coef) > COEF_THRESHOLD]
            formula = ' '.join([f"{coef:+.6f} * {feat_name}" for feat_name, coef in feat_coef])
            score = f"label = {model.intercept_[0]} {'+' if formula[0] != '-' else ''} {formula}"
            print(score)
        else:
            r = plot_feature_importance(model, feature_names=list(model.feature_names_in_), show=False)
            if r:
                plt.savefig(os.path.join(save_dir, f"{save_name}_fimp.svg"), bbox_inches='tight')
        plt.show()
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"可视化{model.__class__.__name__}遇到:{e}")